In [2]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """

    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata["h"] = h
            # update_all is a message passing API.
            g.update_all(
                message_func=fn.copy_u("h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)
        
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [5]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]


def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(200):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print(
                "In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})".format(
                    e, loss, val_acc, best_val_acc, test_acc, best_test_acc
                )
            )


model = Model(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


e:\anaconda3\envs\NLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In epoch 0, loss: 1.952, val acc: 0.162 (best 0.162), test acc: 0.149 (best 0.149)
In epoch 5, loss: 1.887, val acc: 0.226 (best 0.226), test acc: 0.210 (best 0.210)
In epoch 10, loss: 1.763, val acc: 0.166 (best 0.308), test acc: 0.158 (best 0.279)
In epoch 15, loss: 1.572, val acc: 0.178 (best 0.308), test acc: 0.190 (best 0.279)
In epoch 20, loss: 1.316, val acc: 0.274 (best 0.308), test acc: 0.286 (best 0.279)
In epoch 25, loss: 1.009, val acc: 0.480 (best 0.480), test acc: 0.472 (best 0.472)
In epoch 30, loss: 0.692, val acc: 0.648 (best 0.648), test acc: 0.653 (best 0.653)
In epoch 35, loss: 0.427, val acc: 0.704 (best 0.704), test acc: 0.739 (best 0.739)
In epoch 40, loss: 0.244, val acc: 0.722 (best 0.724), test acc: 0.753 (best 0.754)
In epoch 45, loss: 0.136, val acc: 0.720 (best 0.724), test acc: 0.756 (best 0.754)
In epoch 50, loss: 0.078, val acc: 0.738 (best 0.738), test acc: 0.763 (best 0.763)
In epoch 55, loss: 0.048, val acc: 0.746 (best 0.746), test acc: 0.767 (best 0

In [6]:
class WeightedSAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model with edge weights.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """

    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h, w):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        with g.local_scope():
            g.ndata["h"] = h
            g.edata["w"] = w
            g.update_all(
                message_func=fn.u_mul_e("h", "w", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [7]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges(), 1).to(g.device))
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges(), 1).to(g.device))
        return h


model = Model(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.951, val acc: 0.156 (best 0.156), test acc: 0.144 (best 0.144)
In epoch 5, loss: 1.865, val acc: 0.466 (best 0.466), test acc: 0.446 (best 0.446)
In epoch 10, loss: 1.704, val acc: 0.432 (best 0.466), test acc: 0.434 (best 0.446)
In epoch 15, loss: 1.467, val acc: 0.564 (best 0.564), test acc: 0.581 (best 0.581)
In epoch 20, loss: 1.173, val acc: 0.632 (best 0.632), test acc: 0.635 (best 0.635)
In epoch 25, loss: 0.859, val acc: 0.684 (best 0.684), test acc: 0.697 (best 0.697)
In epoch 30, loss: 0.574, val acc: 0.746 (best 0.746), test acc: 0.745 (best 0.745)
In epoch 35, loss: 0.353, val acc: 0.762 (best 0.762), test acc: 0.768 (best 0.768)
In epoch 40, loss: 0.207, val acc: 0.762 (best 0.766), test acc: 0.771 (best 0.773)
In epoch 45, loss: 0.121, val acc: 0.752 (best 0.766), test acc: 0.772 (best 0.773)
In epoch 50, loss: 0.073, val acc: 0.750 (best 0.766), test acc: 0.771 (best 0.773)
In epoch 55, loss: 0.047, val acc: 0.750 (best 0.766), test acc: 0.771 (best 0

In [8]:
def u_mul_e_udf(edges):
    return {"m": edges.src["h"] * edges.data["w"]}

In [9]:
def mean_udf(nodes):
    return {"h_N": nodes.mailbox["m"].mean(1)}